In [4]:
import json
import pickle
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering, AffinityPropagation, SpectralClustering
from sklearn.decomposition import TruncatedSVD

from tqdm.auto import tqdm
from pathlib import Path


In [5]:
N_COMPONENTS = 2
N_CLUSTERS = 10

In [6]:
def reduction_dim(dict_lex, n_components=4):
    dict_lex_reduced = {}
    for k, v in dict_lex.items():
        svd = TruncatedSVD(n_components=n_components)
        svd.fit(np.array([e[1] for e in v]))
        dict_lex_reduced[k] = [(e[0], svd.transform([e[1]])[0]) for e in v]
    return dict_lex_reduced


In [7]:
path = Path('explore_clusters')
path.mkdir(exist_ok=True)


In [8]:
with open('collection_textes.pickle', 'rb') as f:
    collection_textes = pickle.load(f)


In [9]:
alllex = [e.lexicalites for e in collection_textes]

allecarts = [e.ecarts for e in collection_textes]

In [10]:
dict_lex = {taile: [] for taile in range(1000)}
for i, lex in enumerate(alllex):
    dict_lex[len(lex)].append((i, lex))

dict_lex = {k: v for k, v in dict_lex.items() if v}

dict_ecarts = {taile: [] for taile in range(1000)}
for i, ecart in enumerate(allecarts):
    dict_ecarts[len(ecart)].append((i, ecart))

dict_ecarts = {k: v for k, v in dict_ecarts.items() if v}

In [11]:
dict_lex_sup_n_sub = {k: v for k, v in dict_lex.items() if N_COMPONENTS < k < 200}
dict_ecarts_sup_n_sub = {k: v for k, v in dict_ecarts.items() if N_COMPONENTS < k < 200}

In [12]:
dict_lex_reduced = reduction_dim(dict_lex_sup_n_sub, n_components=N_COMPONENTS)
dict_ecarts_reduced = reduction_dim(dict_ecarts_sup_n_sub, n_components=N_COMPONENTS)

/home/marceau/PycharmProjects/memoire/venv/lib64/python3.11/site-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/marceau/PycharmProjects/memoire/venv/lib64/python3.11/site-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/marceau/PycharmProjects/memoire/venv/lib64/python3.11/site-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/marceau/PycharmProjects/memoire/venv/lib64/python3.11/site-packages/sklearn/decomposition/_truncated_svd.py:268: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = exp_var / full_var
/home/marceau/PycharmProjects/memoire/venv/lib64/python3.11/site-packages/sklearn/decomposition/_tru

In [13]:
liste_lex = [e for v in dict_lex_reduced.values() for e in v if len(e[1]) == 2]
liste_ecarts = [e for v in dict_ecarts_reduced.values() for e in v if len(e[1]) == 2]
combined = [(*e, *f) for e, f in zip(liste_lex, liste_ecarts)]

In [30]:
dict_vects = {
    "lex" : liste_lex,
    "ecarts" : liste_ecarts,
    "combined" : combined
}

tests = (
    (5, "lex"),
    (7, "lex"),
    (5, "ecarts"),
    (12, "ecarts"),
    (4, "combined"),
    (6, "combined"),
)

for n_clusters, type_vect in tests:
    km = KMeans(n_clusters=n_clusters, random_state=0, n_init="auto")
    km.fit([e[1] for e in dict_vects[type_vect]])
    dict_clusters = {i: [] for i in range(n_clusters)}

    for i, e in enumerate(dict_vects[type_vect]):
        dict_clusters[km.labels_[i]].append(e[0])

    with open(path / f"clusters_{type_vect}_{n_clusters}.json", "w") as f:
        json.dump(dict_clusters, f, indent=4, ensure_ascii=False)

    exemples = [
        (cluster, collection_textes[value[0]].header, collection_textes[value[0]].texte) for cluster, value in dict_clusters.items()
    ]

    with open(path / f"exemples_{type_vect}_{n_clusters}.json", "w") as f:
        json.dump(exemples, f, indent=4, ensure_ascii=False)


In [29]:
for cluster, value in dict_clusters.items():
    print(cluster, collection_textes[value[0]].header, collection_textes[value[0]].texte)
    1/0


0 {'form': 'vers', 'genre': ['entretien', 'dialogue'], 'handwritten_note': False, 'table_of_content': False, 'illustration': False, 'subject': ['Emprisonnement des\n            princes', 'Longueville II, Henri (1595-1663 ; duc de)', 'Conti, Armand de Bourbon (1629-1666\xa0; prince de)', 'Condé, Louis de Bourbon (1621-1686 ; prince de)', 'Paris (France)'], 'creation': '1650-01-18', 'change': {'@status': 'corrected', '@when': '2022-02-10', '@who': 'AB'}, 'titre': 'Les entretiens mystérieux des trois princes en cage dans\n          le bois de Vincennes, sous les figures du lyon, du renard et du singe, dialogue.', 'dates': {'@type': 'file_creation', '@when': '2022-01-31', '#text': '31 janvier 2022'}, 'langue': 'fra', 'fichier': 'Moreau1256_MAZ.xml', 'path': 'Corpus/Mazarinades/1201-1300/Moreau1256_MAZ.xml', 'json_path': 'Corpus/Mazarinades_jsons/1201-1300/Moreau1256_MAZ.json', 'langue_detectee': 'LGERM', 'lexicalites': [0.9411764705882353, 0.7586206896551724, 0.868421052631579, 0.8, 0.8679

ZeroDivisionError: division by zero